In [ ]:
import numpy as np
from datetime import datetime, timezone
from orbit_util import Orbit_2body, OrbitVisualizer

a = 6878.137
e = 0
i = 97.4
RAAN = 0
w = 0
theta = 0

orb = Orbit_2body()
mu = orb.mu  # Earth's gravitational parameter

# specific angular momentum (h)
h = np.sqrt(mu * a)

# Convert Keplerian to Cartesian coordinates
R0, V0 = orb.keplerian_to_cartesian(e=e, h=h, theta=theta, i=i, RAAN=RAAN, w=w, degree_mode=True)

# Define simulation parameters
T = 10 * 24 * 3600
time_step = 60
scenario_epoch = datetime.now(timezone.utc)

# Simulation
# Two-Body Propagator
sol_2body, t_2body = orb.propagate_init_cond(T=T, time_step=time_step, R0=R0, V0=V0)

# J2 Propagator
sol_j2, t_j2 = orb.propagte_with_J2(T=T, time_step=time_step, R0=R0, V0=V0)

# HPOP Propagator
sol_hpop, t_hpop = orb.HFOP(T=T, time_step=time_step, R0=R0, V0=V0, scenario_epoch=scenario_epoch)

# position vectors
r_2body = sol_2body[:, 0:3]
r_j2 = sol_j2[:, 0:3]
r_hpop = sol_hpop[:, 0:3]

# Calculate ground tracks
lat_2body, long_2body = orb.lat_long_from_ECI(r_eci=r_2body, t=t_2body, scenario_epoch=scenario_epoch)
lat_j2, long_j2 = orb.lat_long_from_ECI(r_eci=r_j2, t=t_j2, scenario_epoch=scenario_epoch)
lat_hpop, long_hpop = orb.lat_long_from_ECI(r_eci=r_hpop, t=t_hpop, scenario_epoch=scenario_epoch)


viz = OrbitVisualizer()
# axis limits(LEO)
limits = np.array([[8000, -8000], [8000, -8000], [8000, -8000]])

# Visualize 3D orbital paths
viz.EarthStatic(r=r_2body, title="Two-Body Orbit (3D)", names=["Two-Body"], colors=["#FF0000"], limits=limits)
viz.EarthStatic(r=r_j2, title="J2 Orbit (3D)", names=["J2"], colors=["#00FF00"], limits=limits)
viz.EarthStatic(r=r_hpop, title="HPOP Orbit (3D)", names=["HPOP"], colors=["#0000FF"], limits=limits)

# Visualize combined ground tracks
latitudes = np.vstack([lat_2body, lat_j2, lat_hpop])
longitudes = np.vstack([long_2body, long_j2, long_hpop])
viz.ground_track(latitudes=latitudes, longitudes=longitudes, names=["Two-Body", "J2", "HPOP"])

print("Orbit simulations and visualizations completed successfully!")

./kernels/naif0012.tls was loaded successfully
./kernels/de440.bsp was loaded successfully


Orbit simulations and visualizations completed successfully!
